Spam: 
Credit: 1032114

### 配置文件验证

In [ ]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config_Spam.yaml') # 读取配置文件

In [ ]:
import os
work_path = os.getcwd()
hparams.pro_type_file = os.path.join(work_path, hparams.pro_type_file)

### 得到数据集相关属性

In [ ]:
import pickle
import pandas as pd
import numpy as np
import torch
# data_pd = pd.read_csv('Letter_train.csv')
# col_names = list(data_pd)
# df[col_name].value_counts()
# pro_types = [('discrete',16) for _ in col_names]
# with open('Letter_pro_type.pkl', 'wb') as f:
#     pickle.dump(pro_types, f)

### dataloader验证

In [13]:
import torch
from dataModule.dataset1 import FlatDataset, ValidDataset
from torch.utils.data import DataLoader

In [15]:
train_dataset = FlatDataset('Credit_Card/Credit模拟缺失_0_3.csv', 
                    pro_type_file='Credit_Card/Credit_Card_pro_type.pkl',
                    replace_dict_file='Credit_Card/Credit_Card数值置换表.pkl')

dataloader = DataLoader(train_dataset, 
                             batch_size=10, 
                             collate_fn=train_dataset.collater, 
                             shuffle=False,
                             num_workers=2)
batch = None
for i in dataloader:
    batch = i
    break

#### 测试集检测

In [ ]:
ValidDataset = ValidDataset(miss_file='Credit_Card/Credit模拟缺失_0_3.csv', 
                            complete_file='Credit_Card/Credit_Card_without_label.csv',
                            pro_type_file='Credit_Card/Credit_Card_pro_type.pkl',
                            replace_dict_file='Credit_Card/Credit_Card数值置换表.pkl')

valid_dataloader = DataLoader(ValidDataset, 
                             batch_size=10, 
                             collate_fn=ValidDataset.collater, 
                             shuffle=False,
                             num_workers=2)
val_batch = None
for i in valid_dataloader:
    val_batch = i
    break

### 模型测试

In [1]:
from modelModule.model import VAE
import torch
import pickle

model = VAE(dim=23, 
            pro_types=pickle.load(open('Credit_Card/Credit_Card_pro_type.pkl', 'rb')),
            replace_dict=pickle.load(open('Credit_Card/Credit_Card数值置换表.pkl', 'rb')))

In [ ]:
global_normal, portion_normal = val_batch['global_normal'], val_batch['portion_normal']
M_matrix = val_batch['miss_matrix']
out, D_list, mu, log_var = model(portion_normal, M_matrix)

In [19]:
global_normal, portion_normal = batch['global_normal'], batch['portion_normal']
M_matrix = batch['miss_matrix']
out, D_list, mu, log_var = model(portion_normal, M_matrix)

### 模型预测输出测试

In [2]:
import pandas as pd
miss_data = pd.read_csv("Credit_Card/Credit模拟缺失_0_3.csv")
model.inference(miss_data)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23
0,20000.0000,1,2,1,24.000000,2,2,-1,-1,-2,...,689.00000,0.0000,422139.56250,0.00000,252571.093750,847197.500,0.00000,0.0000,213124.296875,0.000
1,120000.0000,2,2,0,50.095112,3,2,0,0,0,...,353114.03125,362709.6875,3455.00000,3261.00000,253493.250000,1000.000,1000.00000,1000.0000,0.000000,2000.000
2,90000.0000,2,2,2,50.132286,0,0,0,0,0,...,13559.00000,14331.0000,424672.31250,15549.00000,1518.000000,1500.000,1000.00000,264960.0625,1000.000000,265641.625
3,505857.9375,2,2,1,37.000000,0,0,0,0,6,...,49291.00000,28314.0000,424084.06250,29547.00000,253294.859375,2019.000,1200.00000,1100.0000,1069.000000,1000.000
4,50000.0000,1,2,1,57.000000,-1,0,-1,0,4,...,35835.00000,20940.0000,19146.00000,19131.00000,2000.000000,842876.000,450919.53125,264814.3125,689.000000,679.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000.0000,1,2,1,39.000000,0,0,0,-2,0,...,349679.96875,88004.0000,31237.00000,15980.00000,8500.000000,20000.000,5003.00000,3047.0000,5000.000000,1000.000
29996,150000.0000,2,3,2,50.097469,-1,7,2,3,0,...,3502.00000,364370.8125,5190.00000,0.00000,253813.031250,3526.000,448369.50000,129.0000,0.000000,0.000
29997,30000.0000,1,2,2,37.000000,4,7,-2,-1,0,...,348365.62500,20878.0000,423248.21875,19357.00000,0.000000,0.000,22000.00000,4200.0000,214857.328125,3100.000
29998,80000.0000,1,3,3,41.000000,1,-1,2,-2,0,...,76304.00000,52774.0000,11855.00000,304977.96875,85900.000000,846491.625,1178.00000,1926.0000,52964.000000,264666.000


### 损失函数测试

In [ ]:
from modelModule.loss_function import vae_loss

In [ ]:
MSE_loss, EntropyLoss = vae_loss(portion_normal, out, M_matrix, D_list, model.attribute_type)

In [ ]:
MSE_loss

In [ ]:
EntropyLoss

In [ ]:
a = []
a.append(None)
a.append(None)

In [ ]:
a

In [ ]:
val_batch['global_normal'][1]

In [ ]:
val_batch['portion_normal'][1]

In [ ]:
val_batch['miss_matrix'][1]

In [ ]:
train_dataloader = DataLoader(train_dataset, 
                             batch_size=10, 
                             collate_fn=train_dataset.collater, 
                             shuffle=False,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [ ]:
batch['src_data']

In [ ]:
batch["global_normal"]

In [ ]:
batch["portion_normal"][0]

In [ ]:
a = torch.randint(0,10, size=(5,4,3)).float()
print(a)
f = torch.nn.Softmax(dim=1)
f(a)


In [ ]:
a = torch.randint(0,10,size=(5,12,4)).float()
print(a)
model = torch.nn.Linear(4,10)
model(a[:,1,:]).shape

In [ ]:
model(a[:,0].reshape(-1,1,1)).permute(0, 2, 1).shape

In [ ]:
a = torch.randint(0,10,size=(5,3)).float()
model = torch.nn.Conv1d(in_channels=1, out_channels=8, kernel_size=1, stride=1)
print(a)
embedding_out = torch.tensor([])
embedding_out = torch.cat((embedding_out, model(a[:,0].reshape(-1,1,1)).permute(0, 2, 1)), dim = 1)
embedding_out = torch.cat((embedding_out, model(a[:,1].reshape(-1,1,1)).permute(0, 2, 1)), dim = 1)
print(embedding_out.shape)
embedding_out

### 模型精度测定

In [ ]:
from dataModule.dataset1 import FlatDataset
from modelModule.model5 import VAE5
from torch.utils.data import DataLoader
import torch
import numpy as np
import pandas as pd
import pickle

In [ ]:
## 模型参数载入
model = VAE5(dim=9, 
            pro_types=pickle.load(open('Breast\Breast_pro_type.pkl', 'rb')),
            replace_dict=pickle.load(open('Breast\Breast数值置换表.pkl', 'rb')))# nhead=3
# checkpoint_path = r'模型参数保存/Credit_Card_model5_norm_way_minmax_norm_/version_1/checkpoints/epoch=31-step=3008.ckpt'    
data_norm = 'minmax_norm'  # 数据集采用正则化方式minmax_norm,mean_norm

# checkpoint = torch.load(checkpoint_path)
# model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
# model.eval()

print('载入成功')

In [ ]:
miss_file = 'Breast\Breast模拟缺失_0_3.csv'      # 缺失数据集
miss_data = pd.read_csv(miss_file)

c, _ = model.inference(miss_data)


In [ ]:
c.shape

In [ ]:
# ## 模型参数载入
# model = VAE5(dim=23, pro_types=pickle.load(open('Credit_Card\Credit_Card_pro_type.pkl', 'rb'))) # nhead=3
# checkpoint_path = r'模型参数保存/Credit_Card_model5_norm_way_minmax_norm_/version_1/checkpoints/epoch=31-step=3008.ckpt'    
# data_norm = 'minmax_norm'  # 数据集采用正则化方式minmax_norm,mean_norm
# train_csv = r'Credit_Card_model_train.csv'
# test_csv = r'Credit_Card_model_test.csv'

# checkpoint = torch.load(checkpoint_path)
# model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
# model.eval()
# ## 载入训练集的均值(最小)方差(最大)
# train_dataset = FlatDataset(train_csv, data_norm=data_norm, pro_type_file='Credit_Card_pro_type.pkl')
# model.get_global_min_max(dataset=train_dataset)
# print('载入成功')

In [ ]:

# test_dataset = FlatDataset(test_csv, data_norm=data_norm, pro_type_file='Credit_Card_pro_type.pkl')
# data_size = 10
# src_data = np.stack([test_dataset[index]['src_data'] for index in range(data_size)], axis=0)
# normal_data = np.stack([test_dataset[index]['global_normal_data'] for index in range(data_size)], axis=0)
# portion_normal_data = np.stack([test_dataset[index]['portion_normal_data'] for index in range(data_size)], axis=0)
# _, miss_matrix = get_missing(src_data, 0.3) # 得到缺失矩阵
# input = portion_normal_data.copy()
# input[miss_matrix == 0] = np.nan

In [ ]:
# imputed_data, output = model.inference(input, miss_matrix) # inference输出的结果已经重构后的结果
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()

In [ ]:
# print(normal_data[0])
# print(output[0])


In [ ]:
imputed_data[1]

In [ ]:
result_show(src_data, imputed_data, miss_matrix)

In [ ]:
imputed_data

In [ ]:
imputed_data.shape

In [ ]:
# test_dataset = FlatDataset('Letter_test.csv', data_norm='mean_norm') # minmax_norm
# index = 102
# src_data, M_matrix, miss_data = test_dataset[index]['src_data'].reshape(1,-1), test_dataset[index]['global_miss_matrix'].reshape(1,-1), test_dataset[index]['global_miss_data'].reshape(1,-1)
# src_data = test_dataset[index]['src_data'].reshape(1,-1)
# normal_data = test_dataset[index]['global_normal_data'].reshape(1,-1)
# miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

# imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
# res_imputed_data = np.around(restore_data(imputed_data, test_dataset.Max_Val, test_dataset.Min_Val))
# result_show(src_data, res_imputed_data, M_matrix)

In [ ]:
# batch测试
# src_data, miss_data, M_matrix = batch['src_data'], batch['miss_data'], batch['miss_matrix']
# normal_data = batch['normal_data']
# imputed_data, _, _ = model(miss_data, M_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
# res_imputed_data = np.around(restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val))